In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-03-21


Found rundate 2024-03-20 10:00:02 < 2024-03-21 at commit 54a6d93f; returning commit 1c212b1


'2024-03-21 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-03-21 10:00:02, NJSP was reporting 130 YTD deaths
Current YTD Deaths (2025-03-21 10:00:01-04:00): 106
Previous year YTD Deaths (adjusted; 2024-03-21 10:00:02): 128.7342964446226
Projected 2025 total: 645.7107695640061
-17.7% change


FAUQStats cache miss: 2024, Thu Mar 21 10:00:02 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

CCODE       CNAME MCODE              MNAME                   STREET  \
ACCID                                                                       
12918    12   Middlesex  1218    Plainsboro Twsp                      NaN   
12923    11      Mercer  1111            Trenton                      NaN   
12932    01    Atlantic  0112      Hamilton Twsp                      NaN   
12933    02      Bergen  0248        Ramsey Boro              East Oak St   
12927    20       Union  2009        Linden City                      NaN   
...     ...         ...   ...                ...                      ...   
13063    10   Hunterdon  1012  Glen Gardner Boro                      NaN   
13064    03  Burlington  0306    Burlington Twsp                      NaN   
13070    11      Mercer  1102         Ewing Twsp                      NaN   
13071    20       Union  2004     Elizabeth City                      NaN   
13072    20       Union  2009        Linden City  N Wood Ave/E Gibbons St   

                               LOCATION  FATALITIES  FATAL_D  FATAL_P  \
ACCID                                                                   
12918                   State Highway 1         1.0      0.0      0.0   
12923                 State Highway 129         1.0      1.0      0.0   
12932  Atlantic City Expressway MP 20.5         1.0      1.0      0.0   
12933                       East Oak St         1.0      1.0      0.0   
12927                  State Highway 27         1.0      0.0      0.0   
...                                 ...         ...      ...      ...   
13063          State Highway 31 MP 37.3         1.0      0.0      0.0   
13064            Interstate 295 MP 46.8         1.0      0.0      0.0   
13070                        County 611         1.0      1.0      0.0   
13071           State Highway 1 MP 45.6         1.0      0.0      0.0   
13072           N Wood Ave/E Gibbons St         1.0      1.0      0.0   

       FATAL_T  FATAL_B HIGHWAY  INJURIES                        dt  
ACCID                                                                
12918      1.0      0.0       1       NaN 2024-01-01 01:56:00-05:00  
12923      0.0      0.0     129       NaN 2024-01-03 12:14:00-05:00  
12932      0.0      0.0     446       NaN 2024-01-05 09:14:00-05:00  
12933      0.0      0.0     NaN       NaN 2024-01-05 14:12:00-05:00  
12927      1.0      0.0      27       NaN 2024-01-06 08:37:00-05:00  
...        ...      ...     ...       ...                       ...  
13063      1.0      0.0      31       NaN 2024-03-16 22:01:00-04:00  
13064      1.0      0.0     295       0.0 2024-03-17 08:15:00-04:00  
13070      0.0      0.0     611       NaN 2024-03-18 06:37:00-04:00  
13071      1.0      0.0       1       0.0 2024-03-18 15:06:00-04:00  
13072      0.0      0.0     NaN       NaN 2024-03-20 14:36:00-04:00  

[125 rows x 14 columns]

Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

driver  passenger  pedestrian  cyclist  crashes
county                                                     
Atlantic         3          0           2        0        5
Bergen           5          1           1        0        7
Burlington       5          1           2        0        8
Camden           9          0           2        0       11
Cape May         2          0           0        0        2
Cumberland       3          1           2        0        5
Essex            0          0           3        1        4
Gloucester       5          2           0        0        5
Hudson           0          0           2        0        2
Mercer           1          0           3        0        4
Middlesex        4          1           1        0        6
Monmouth         2          1           0        0        2
Morris           1          0           0        0        1
Ocean            8          2           4        0       12
Passaic          1          1           3        1        6
Salem            1          0           1        0        2
Somerset         4          2           0        0        6
Sussex           0          1           0        0        1
Union            2          1           3        0        6
Warren           1          3           1        0        5

Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

21.7% through the year, 78.3% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                              
Atlantic   driver         3       31        3
Bergen     driver         5       14        5
Burlington driver         7       32        5
Camden     driver         4       19        9
Cape May   driver         1        8        2
...                     ...      ...      ...
Salem      crashes        2       14        2
Somerset   crashes        1       18        6
Sussex     crashes        1       12        1
Union      crashes        4       35        6
Warren     crashes        3       14        5

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type        crashes  cyclist  driver  passenger  pedestrian
county                                                     
Atlantic         47        2      31          1          16
Bergen           31        1      14          5          13
Burlington       42        1      28          3          12
Camden           40        0      28          2          11
Cape May         12        0      11          0           1
Cumberland       38        0      29          6           9
Essex            44        2      14          6          26
Gloucester       29        0      20         11           5
Hudson           15        1       7          0           9
Hunterdon         5        0       3          2           1
Mercer           28        1      10          4          17
Middlesex        42        2      22          9          12
Monmouth         35        4      16          3          13
Morris           16        1       9          1           5
Ocean            50        2      37          7          11
Passaic          31        2      12          3          20
Salem            14        2       7          2           3
Somerset         41        0      21          4           6
Sussex           12        1       5          3           2
Union            40        1      21          7          14
Warren           18        0       9          6           3

In [9]:
projected.to_csv(PROJECTED_CSV)